In [ ]:
import numpy as np
from cmdstanpy import cmdstan_path, CmdStanModel
from pathlib import Path
import matplotlib.pyplot as plt
import json
import pandas as pd

## Beer cooling example

At time t=0, a glass of beer is dropped to a bucket of cold water with given temperature. Beer is cooled by water and heated with the outside air that is at a given temperature. Beer temperature is measured every minute, and the goal is to infer the heat transfer coefficients through the glass and air, plus the model and observation noise standard deviations.

We write the model as the following DLM system:

<img src="https://latex.codecogs.com/svg.image?A&space;=&space;\begin{bmatrix}1-\theta_1&space;-&space;\theta_2\end{bmatrix},&space;\&space;B&space;=&space;&space;\begin{bmatrix}\theta_1&space;&&space;\theta_2&space;\end{bmatrix},&space;\&space;C&space;=&space;&space;\begin{bmatrix}1&space;\end{bmatrix},&space;\&space;R&space;=&space;\theta_3^2,&space;\&space;Q&space;=&space;\theta_4^2" title="A = \begin{bmatrix}1-\theta_1 - \theta_2\end{bmatrix}, \ B = \begin{bmatrix}\theta_1 & \theta_2 \end{bmatrix}, \ C = \begin{bmatrix}1 \end{bmatrix}, \ R = \theta_3^2, \ Q = \theta_4^2" align="left"/>

### 1) Define the functions that build the needed matrices

Here, we write a Stan functions block that builds the matrices given the parameters. Note that the noise parameters are separated from the "other" parameters so that we can assign priors separately for them (e.g. enforce positivity).

In [ ]:
beer_functions = """
functions {
    matrix build_A(vector theta) {
        matrix[1,1] A;
        A[1,1] = 1-theta[1]-theta[2];
        return A;
    }
    matrix build_B(vector theta) {
        matrix[1,2] B;
        B[1,1] = theta[1];
        B[1,2] = theta[2];
        return B;
    }
    matrix build_C(vector theta) {
        matrix[1,1] C;
        C[1,1] = 1;
        return C;
    }
    matrix build_Q(vector noise_theta) {
        matrix[1,1] Q;
        Q[1,1] = square(noise_theta[1]);
        return Q;
    }
    matrix build_R(vector noise_theta) {
        matrix[1,1] R;
        R[1,1] = square(noise_theta[2]);
        return R;
    }
}
"""

### 2) Compile the code with `CmdStanPy`

First, we read the general DLM stan code and append the user-defined functions to the model file.

In [ ]:
dlm_code = Path('../dlm.stan').read_text()
Path('beer.stan').write_text(beer_functions + dlm_code);
beer_model = CmdStanModel(stan_file='beer.stan')

### 3) Construct the data

The general DLM code always needs the same input data variables. Note that the vectors and matrices need to be given as python lists so that the serialization to JSON works.

In [ ]:
y = [28, 24, 20, 17.5, 15.5, 13.5, 12, 11, 10]

beer_data = {
    'N_obs': len(y),
    'N_theta': 2,
    'N_noise_theta': 2,
    'state_dim': 1,
    'input_dim': 2,
    'obs_dim': 1,
    'Y_obs': [[yi] for yi in y],
    'U_obs': len(y)*[[5, 23]],
    'm0': [31.0],
    'P0': [[1**2]],
    'theta_mu': [0, 0],
    'theta_Sig': [[10**2, 0],[0, 10**2]],
    'noise_mu': [0.2, 0.2],
    'noise_Sig': [[0.2**2, 0], [0, 0.2**2]]
}

In [ ]:
with open('beer_data.json', 'w') as f:
    json.dump(beer_data, f)

### 4) Fit the model with HMC

Save the output files to a separate output folder to keep things clean. Print the fit object to get some details about the files that were produced.

In [ ]:
beer_fit = beer_model.sample(data='beer_data.json', output_dir='output')
print(beer_fit)

### 5) Access the parameters and plot results

Let us draw the sampled states and parameters. Sampled parameters can be accessed with `CmdStanPy`:s helper `stan_variable`.

In [ ]:
draws_noise = beer_fit.stan_variable('noise_theta')
draws_theta = beer_fit.stan_variable('theta')
draws_x = beer_fit.stan_variable('x_samples')

In [ ]:
plt.figure(figsize=(7,4))

plt.plot(range(len(y)), draws_x[0::10,:,0].T, c='0.5', alpha=0.2)
plt.plot(range(len(y)), np.quantile(draws_x[0:,:,0].T, [0.025, 0.975], axis=1).T, 'k--')
plt.plot(range(len(y)), np.quantile(draws_x[0:,:,0].T, 0.5, axis=1), 'k-')
plt.plot(range(len(y)), y, 'r.')

plt.grid(True)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(pd.DataFrame(draws_theta));

In [ ]:
pd.plotting.scatter_matrix(pd.DataFrame(draws_noise));

### Notes on results

- First parameter identified well, the other not (data does not even tell the sign of the coefficient)
- Observation/model Noise parameters are impossible to identify separately, informative prior needed for one of the two